In [9]:
import os
import glob

import xarray as xr

In [10]:
roms_grid_ds = xr.open_dataset('/cluster/projects/nn8103k/A20/Grid/A20niva_grd_v1.nc')
data_folder = '/cluster/projects/nn8103k/A20/FORCING/HINDCAST/v3/ERA5/6hourly/'
cice_folder = '/cluster/projects/nn8103k/A20/FORCING/HINDCAST/v3/ERA5/cice/'
globs_names = (
    ('era5_q_year_*_roms_halo.nc', 'Qair', 'qair_time'),
    ('era5_tp_year_*_roms_halo.nc', 'rain', 'rain_time'),
    ('era5_msl_year_*_roms_halo.nc', 'Pair', 'pair_time'),
    ('era5_t2m_year_*_roms_halo.nc', 'Tair', 'Tair_time'),
    ('era5_u10_year_*_roms_halo.nc', 'Uwind', 'wind_time'),
    ('era5_v10_year_*_roms_halo.nc', 'Vwind', 'wind_time'),
)

In [36]:
var_number = 5

In [37]:
filepaths = sorted(glob.glob(data_folder+globs_names[var_number][0]))
var_name, time_name = globs_names[var_number][1], globs_names[var_number][2]
filepath = filepaths[0]
filename = os.path.basename(filepath)

In [38]:
ds = xr.open_dataset(filepath)

In [39]:
coords = {
    time_name: ds[var_name][time_name], 
    'lon': roms_grid_ds.lon_rho, 
    'lat': roms_grid_ds.lat_rho,
}
ds_cice = ds[var_name].interp(coords)

In [40]:
ds_cice.to_netcdf(cice_folder+filename)